## We run the methods of the paper on additional social networks
### In this notebook, we run it on large online social networks

In [1]:
# reload imported files automatically without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from lib import *
from pathcensus import PathCensus
from pathcensus.nullmodels import UBCM
from pathcensus.inference import Inference
from pathcensus.utils import set_seed
from mandarina.benchmark import timer
import networkx as nx


In [3]:
@timer
def calculate_structural_measures(network_name, g, n_samples_null_model):
    """
    This function calculates the structural measures of a network and 
    its null model average values by fitting a UBCM null model to the network.
    """
    n_total = g.number_of_nodes()  # get total number of nodes
    # remove self-loops and multiedges and get largest connected component
    g = preprocess_graph(g)  ; g
    n_giant = g.number_of_nodes()  # get number of nodes in largest connected component
    degseq = sorted([d for n, d in g.degree()], reverse=True)  # get degree sequence
    dataset = ""
    network_name = network_name.split(".")[0]
    label = "large online"
    
    properties = {
            "idx": 1,
            "dataset": dataset,
            "name": network_name,
            #"graph": [g],  # get graph
            "n_nodes": n_giant,  # get number of nodes in largest connected component
            "frac_total": n_giant
            / n_total,  # get fraction of nodes in largest connected component
            "density": nx.density(g),  # get density
            "dbar": np.mean(degseq),  # get mean degree
            "dcv": np.std(degseq)
            / np.mean(degseq),  # get coefficient of variation of degree
            "dmax": np.max(degseq),  # get maximum degree
        }
    
    model = UBCM(g)  # initialize model
    model.fit()  # fit model
    model.validate()  # validate model
    # compare null model to actual graph using statistics function
    infer = Inference(g, model, statistics)
    data, null = infer.init_comparison(n=n_samples_null_model)
    original_network_values = dict(data)
    original_network_values = {'data_' + k: v for k, v in original_network_values.items()}
    null_model_mean_values = dict(null.mean(numeric_only=True)) 
    null_model_mean_values = {f'null_{n_samples_null_model}_{k}': v for k, v in null_model_mean_values.items()}
    return properties | null_model_mean_values |original_network_values | null_model_mean_values

In [4]:
OUTPUT_CSV_FILE_PATH = RESULT_DIR_PATH / "structural_measures_large_online.csv"

In [5]:
def dataset_size_filter(dataset_path, size):
    """Filter datasets by size, given in bytes."""
    return dataset_path.stat().st_size < size

In [6]:
file_filter = lambda file: file.is_file()
large_online_network_files = list(filter(file_filter, (DATA_DIR_PATH / "online" / "large").glob("**/*")))

filtered_dataset_paths = list(filter(lambda x: dataset_size_filter(x, 50000000), large_online_network_files))

print(len(filtered_dataset_paths))

results = []
for i, file in enumerate(filtered_dataset_paths): # loop over all online large social network files
    f = gml_cleaner(file) # clean gml file
    g = nx.read_gml(f, label="id") # load into networkx
    print("Running calculations for network: ", file.name) 
    # calculate measures from original network and null model
    result = calculate_structural_measures(file.name.split(".")[0] , g, n_samples_null_model=10) 
    #df = pd.DataFrame(result).reset_index()
    #df[df.columns[:-1]].to_csv(OUTPUT_CSV_FILE_PATH, mode='a', header=not OUTPUT_CSV_FILE_PATH.is_file()) # append result row to csv file
    results.append(result) # collect results


33
Running calculations for network:  NYC_restaurant_tips.gml
Function calculate_structural_measures with args (('NYC_restaurant_tips', <networkx.classes.multigraph.MultiGraph object at 0x7f626ad56110>), {'n_samples_null_model': 10}) took: 14.3793 seconds.
Running calculations for network:  NYC_restaurant_checkin.gml
Function calculate_structural_measures with args (('NYC_restaurant_checkin', <networkx.classes.multigraph.MultiGraph object at 0x7f6268afcd30>), {'n_samples_null_model': 10}) took: 3.0401 seconds.
Running calculations for network:  douban.gml
Function calculate_structural_measures with args (('douban', <networkx.classes.multigraph.MultiGraph object at 0x7f6267cddcc0>), {'n_samples_null_model': 10}) took: 1144.8680 seconds.
Running calculations for network:  old.gml
Function calculate_structural_measures with args (('old', <networkx.classes.multigraph.MultiGraph object at 0x7f6268e5f400>), {'n_samples_null_model': 10}) took: 537.0661 seconds.
Running calculations for networ

In [7]:
df = pd.concat([pd.DataFrame(result) for result in results]).reset_index() # combine all data to dataframe for analysis
df = df[df.columns[:-1]]

In [8]:
df

,_,idx,dataset,name,n_nodes,frac_total,density,dbar,dcv,dmax,...,null_10_sim_e,null_10_comp_g,null_10_comp,null_10_comp_e,data_sim_g,data_sim,data_sim_e,data_comp_g,data_comp,data_comp_e
0,0,1,,NYC_restaurant_tips,5372,0.838066,0.000614,3.295607,1.569503,196,...,0.003298,0.004624,0.001359,0.002543,0.000000,0.000000,0.000000,0.014416,0.003092,0.006515
1,0,1,,NYC_restaurant_checkin,4906,0.993922,0.001118,5.485936,1.044863,88,...,0.003444,0.004544,0.002013,0.003177,0.000000,0.000000,0.000000,0.011630,0.005808,0.008769
2,0,1,,douban,154908,1.000000,0.000027,4.223952,2.781348,287,...,0.001298,0.002043,0.000267,0.001270,0.010373,0.001901,0.007750,0.006644,0.000755,0.003354
3,0,1,,old,105091,0.919238,0.000065,6.811639,2.068222,692,...,0.000882,0.001637,0.000334,0.000828,0.115086,0.073430,0.123549,0.030024,0.007786,0.018127
4,0,1,,gplus_106382433884876652170,4903,1.000000,0.060190,295.052417,1.053766,2476,...,0.202768,0.080680,0.054151,0.077262,0.239926,0.145436,0.219069,0.110713,0.072958,0.106142
5,0,1,,gplus_106228758905254036967,4839,0.999380,0.022479,108.752222,1.153716,1233,...,0.090982,0.065138,0.036881,0.057237,0.301261,0.213295,0.310858,0.060981,0.030823,0.049776
6,0,1,,gplus_116931379084245069738,4829,0.999586,0.038502,185.889832,1.072778,1737,...,0.137314,0.079559,0.048874,0.072868,0.217978,0.141611,0.202476,0.094009,0.055424,0.084470
7,0,1,,gplus_112573107772208475213,3794,0.998684,0.008368,31.738535,1.780662,548,...,0.085765,0.052001,0.020899,0.044149,0.174692,0.083500,0.169532,0.060478,0.023330,0.050768
8,0,1,,gplus_115455024457484679647,3464,0.992550,0.018095,62.663395,1.815547,919,...,0.182763,0.049082,0.022741,0.047532,0.284835,0.112724,0.259228,0.039766,0.023916,0.045025
9,0,1,,gplus_107040353898400532534,4369,1.000000,0.071511,312.361639,0.948654,2185,...,0.209179,0.091272,0.063005,0.087199,0.257560,0.169155,0.235457,0.121351,0.081376,0.116609


In [9]:
df.to_csv(RESULT_DIR_PATH / "structural_measures_large_online.csv", index=False)